In [9]:
# torch는 딥러닝을 구성하기 편하도록 META에서 만든 딥러닝 라이브러리, 텐서플로우랑 똑같은 딥러닝 라이브러리 # TMI=> torch 연구직에서 주로썼고, tensorflow 필드에서 주로썼었는데, 암튼 토치로 넘어오는 추세
import torch

# 신경망을 구축하기 쉽게 해주는 함수
import torch.nn as nn

In [10]:
# GPU 확인  , gpu 가 없으면 cpu로 연산, gpu가 있으면 gpu로 연산하겠다는 준비, 따라서 device는 cpu나 gpu 문자열로 바뀐다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [11]:
# 4명의 데이터가 있는데, 1명당 2개의 컬럼을 가지고 있다. 그리고 그걸 2차원 리스트로 4명의 데이터를 선언 후
# 토치가 알아먹을 수 있는 Tensor 타입으로 바꾼다. (이때 tensor type은 float)
# to(device)는 모델이 gpu에서 연산 될 것이기 때문에 to(cuda)를 함으로써 데이터를 gpu에 얹어 놓는다.

X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

In [13]:
model = nn.Sequential( # 다른 라이브러리보다 유연한 설계가 가능, 다른 라이브러리는 비교적 간단해서 선호됨
          nn.Linear(2, 10, bias=True), # input_layer = 2, hidden_layer1 = 10                  # 2개의 노드, 아웃풋은 10개
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), # hidden_layer1 = 10, hidden_layer2 = 10                   
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), # hidden_layer2 = 10, hidden_layer3 = 10
          nn.Sigmoid(),
          nn.Linear(10, 1, bias=True), # hidden_layer3 = 10, output_layer = 1
          nn.Sigmoid()
          ).to(device)

In [14]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

In [15]:
# GPU 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [16]:
# 5명의 데이터, 각 데이터는 3개의 컬럼(피처)를 갖는다.
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
# 정답값은 1사람당 1개
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [17]:
# X 데이터 셋과 Y 데이터 셋을 포장함
dataset = TensorDataset(x_train, y_train)
print(list(dataset))

[(tensor([73., 80., 75.]), tensor([152.])), (tensor([93., 88., 93.]), tensor([185.])), (tensor([89., 91., 90.]), tensor([180.])), (tensor([ 96.,  98., 100.]), tensor([196.])), (tensor([73., 66., 70.]), tensor([142.]))]


In [18]:
# 2개 데이터를 1세트로 만들어서 전체 데이터 셋을 쪼갠다. 이때, 만든 n개의 세트들의 순서를 shuffle 한다.
dataloader = DataLoader(dataset, batch_size=2, shuffle=True) # shuffle => 개개개 고양이고양이 이런걸 셔플해서 섞어줌

In [19]:
model = nn.Sequential(
          nn.Linear(3, 10, bias=True), # input_layer = 2, hidden_layer1 = 10
          nn.ReLU(),
          nn.Linear(10, 10, bias=True), # hidden_layer1 = 10, hidden_layer2 = 10
          nn.ReLU(),
          nn.Linear(10, 10, bias=True), # hidden_layer2 = 10, hidden_layer3 = 10
          nn.ReLU(),
          nn.Linear(10, 1, bias=True), # hidden_layer3 = 10, output_layer = 1
          nn.ReLU()
          ).to(device)

In [20]:
criterion = torch.nn.MSELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) # 0.00001

In [ ]:
nb_epochs = 5  # 전체학습때 5라면 ,미니배치때는 줄여봐야함
for epoch in range(nb_epochs + 1): # 전체 데이터 셋으로 5번 학습할 것이다.
  for batch_idx, samples in enumerate(dataloader): # minibatch 데이터를 반환하면서 index도 반환한다. (너무 초 대량의 데이터가 들어가면 GPU 터질까봐 minibatch를 한다)

    x_train, y_train = samples 

    x_train= x_train.to(device)
    y_train= y_train.to(y_train)

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = criterion(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

In [22]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [23]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

In [24]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [25]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 433970869.85it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 114171247.71it/s]


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 195817752.89it/s]


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 23260718.89it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [26]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True) # drop_last 마지막 남은 데이터를 버릴거냐 안버릴거냐

In [27]:
# torch.nn.Module:  PyTorch의 모든 Neural Network의 Base Class
class CNN(torch.nn.Module): # 상속

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),   # Conv2d 채널을
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True) # 1개의 데이터에 10개의 클래스 값

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [28]:
# CNN 모델 정의
model = CNN().to(device)

In [29]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음. #CrossEntropyLoss 의 기능 soft max 부터 cross entropy까지 계산해줌
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [30]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [31]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape

        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.225595474
[Epoch:    2] cost = 0.0629839227
[Epoch:    3] cost = 0.0463118628
[Epoch:    4] cost = 0.037405543
[Epoch:    5] cost = 0.0314706899
[Epoch:    6] cost = 0.0260603409
[Epoch:    7] cost = 0.0218301043
[Epoch:    8] cost = 0.0180830602
[Epoch:    9] cost = 0.0162577052
[Epoch:   10] cost = 0.0129458187
[Epoch:   11] cost = 0.00997531693
[Epoch:   12] cost = 0.0103649357
[Epoch:   13] cost = 0.00820257701
[Epoch:   14] cost = 0.0065609361
[Epoch:   15] cost = 0.00653103925


In [32]:
# 학습을 진행하지 않을 것이므로 torch.no_grad(), gradient descent를 하지마라고 명령내리는 것
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
                        # CNN은 10개의 아웃풋으로 각 10개의 클래스에 대한 피처값이 나온다, 이를 axis 1방향으로 max값을 찾는다는 것
                        # 이를 axis 1방향으로 max값을 찾는다는 것
    correct_prediction = torch.argmax(prediction, 1) == Y_test
                        # torch.argmax(prediction, 1)는 피쳐값중 max인 인덱스 (=label)을 뽑는것
   #accuracy = correct_prediction.float().mean()
   #print('Accuracy:', accuracy.item())

/usr/local/lib/python3.9/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.9/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [33]:
# 모델 저장 
torch.save(model.state_dict(),"cnn_model.pt")

In [34]:
# 학습을 진행하지 않을 것이므로 torch.no_grad(), gradient descent를 하지마라고 명령내리는 것

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

with torch.no_grad():

    # 이미지 파일 경로 설정
    img = Image.open("./8.png")
    transform = transforms.Compose([
          transforms.Grayscale(num_output_channels=1), # RGB(3D) -> Gray(2D)
          transforms.Resize((28, 28)), # 모델 인풋에 맞게
          transforms.ToTensor(), # 토치 텐서 타입으로 맞춰줘야함
      ])
    
    img_tensor = transform(img).to(device) # [1, 28, 28]
    img_tensor = img_tensor.unsqueeze(0) # [1, 1, 28, 28] #모델이 원래 [배치사이즈 가로 세로 등]

    print(img_tensor.shape)

    prediction = model(img_tensor)
                        # CNN은 10개의 아웃풋으로 각 10개의 클래스에 대한 피처값이 나온다, 이를 axis 1방향으로 max값을 찾는다는 것 
    print('result:', torch.argmax(prediction, 1))

torch.Size([1, 1, 28, 28])
result: tensor([8], device='cuda:0')
